# Introduction to HydroGenerate


The HydroGenerate (HG) package was created to assist in the process of evaluating hydropower potential from different types of hydropower generation technologies / facilities. HG is an open-source python library that has the capability of estimating hydropower generation for diversion and hydrokinetic projects.

This document shows a quick workflow for working with HG. All the functionality of HG is encapsulated into a single function (calculate_hp_potential) to facilitate HG utilization by users with limited programming experience. All the base code used in each function is available for reuse and modification. Contributions are welcomed, particularly the addition of new methods, expanding the current functionality, or fixing existing issues with HG. 


In [20]:
%load_ext autoreload
%autoreload 2
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import urllib3
urllib3.disable_warnings()

# Local imports
from hat.api_call import *
from hat.utils.hydropower_potential import *
# from hat.utils.PyGeoTools.geolocation import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Example 1. Basic Calculation:

This section presents the workflow for basic hydropower calculations. Basic calculations compute power, head, or flow, given any combination of two parameters. Basic calculations are included for users who do not have data about their system, and it is intended to provide a general idea on existing resources. 

In [2]:
# 1.1) Calculate power from a given head and flow

flow = 8000 # given flow, in cfs
head = 20 # head, in ft
power = None

hp = calculate_hp_potential(flow= flow, head= head, rated_power= power)

print("Hydropower potential is {} kW".format(round(hp.rated_power, 0)))

Hydropower potential is 11515.0 kW


In [3]:
# 1.2) Calculate head from a given head and flow

flow = 8000 # given flow, in cfs
head = None # head, in ft
power = 11515

hp = calculate_hp_potential(flow= flow, head= head, rated_power= power)
print("The head required to obtain {} kW with a flow of {} cfs is {} ft".format(hp.rated_power, hp.flow, round(hp.head,1)))

The head required to obtain 11515 kW with a flow of 8000.0 cfs is 20.0 ft


In [4]:
# 1.3) Calculate flow from a given head and power

flow = None # given flow, in cfs
head = 20 # head, in ft
power = 11515

hp = calculate_hp_potential(flow= flow, head= head, rated_power= power)
print("The flow required to obtain {} kW with a head of {} ft is {} cfs".format(hp.rated_power, hp.head, round(hp.flow, 0)))

The flow required to obtain 11515 kW with a head of 20.0 ft is 8000.0 cfs


### Example 2. Diversion:

A diversion, sometimes called a “run-of-river” facility, channels a portion of a river through a canal and/or a penstock to utilize the natural decline of the river bed elevation to produce energy. A penstock is a closed conduit that channels the flow of water to turbines with water flow regulated by gates, valves, and turbines. A diversion may not require the use of a dam. Check [DOE - Types of Hydropower Plant](https://www.energy.gov/eere/water/types-hydropower-plants#:~:text=There%20are%20three%20types%20of,renewable%20energy%20to%20the%20grid.) for additional infomration. 

In diversion projects, HydroGenerate focuses on calculating hydropwoer under diffrent configurations.

In [6]:
# 2.1) Head, power, and lenght of penstock are known. FLow is a single value.
# In this scenario HydroGenerate will select a turbine, compute effieincy for the given flow
# and values within 0.6 to 1.2 the given flow, penstock diameter (assuming steel if no material is given),
# head loss for all flows, rater power,
# power a given range of flow, 

flow = 8000 # cfs
head = 20 # ft
power = None
penstock_length = 50 # ft
hp_type = 'Diversion'

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length)

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (kW):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nFlow range evaluated (cfs):', np.round(hp.flow, 1))
print('Turbine Efficiency for the given flow range:', np.round(hp.effi_cal,3))
print('Power (Kw) for the given flow range:', np.round(hp.power,1))


Design flow (cfs): 8000.0
Head_loss at design flow (ft): 1.99
Turbine type: Kaplan
Rated Power (kW): 10894.27
Net head (ft): 18.01
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Penstock length (ft): 50.0
Penstock diameter (ft): 16.44
Runner diameter (ft): 5.33

Flow range evaluated (cfs): [4800.  5082.4 5364.7 5647.1 5929.4 6211.8 6494.1 6776.5 7058.8 7341.2
 7623.5 7905.9 8188.2 8470.6 8752.9 9035.3 9317.6 9600. ]
Turbine Efficiency for the given flow range: [0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.91  0.908
 0.904 0.895 0.881 0.858 0.82  0.762]
Power (Kw) for the given flow range: [ 6996.1  7375.7  7748.7  8115.4  8475.7  8829.3  9176.   9515.3  9846.2
 10166.7 10472.4 10755.1 11000.7 11187.  11279.9 11230.5 10970.6 10407.8]


In [7]:
# 2.2) Exploring additional options:
#  Using Hazen-Williams for head loss calculation
flow = 8000 # cfs
head = 20 # ft
power = None
penstock_length = 50 # ft
hp_type = 'Diversion'
headloss_method= "Hazen-Williams"

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length,
                           headloss_method= headloss_method)



# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nFlow range evaluated (cfs):', np.round(hp.flow, 1))
print('Turbine Efficiency for the given flow range:', np.round(hp.effi_cal,3))
print('Power (kW) for the given flow range:', np.round(hp.power,1))

Design flow (cfs): 8000.0
Head_loss at design flow (ft): 2.0
Turbine type: Kaplan
Rated Power (Kw): 10885.57
Net head (ft): 18.0
Generator Efficiency: 0.98
Head Loss method: Hazen-Williams
Penstock length (ft): 50.0
Penstock diameter (ft): 12.24
Runner diameter (ft): 5.33

Flow range evaluated (cfs): [4800.  5082.4 5364.7 5647.1 5929.4 6211.8 6494.1 6776.5 7058.8 7341.2
 7623.5 7905.9 8188.2 8470.6 8752.9 9035.3 9317.6 9600. ]
Turbine Efficiency for the given flow range: [0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.91  0.908
 0.904 0.895 0.881 0.858 0.82  0.762]
Power (kW) for the given flow range: [ 6973.7  7351.9  7723.8  8089.8  8449.8  8803.6  9151.1  9491.7  9824.7
 10148.  10457.4 10744.6 10995.9 11188.7 11289.1 11248.  10996.6 10441.8]


In [8]:
# 2.3) Exploring additional options:
# Using Hazen-Williams for head loss calculation and 
# a concrete penstock

# 1.1) Exploring additional options:
#  Selecting a diffrent material for the penstock
flow = 8000 # cfs
head = 20 # ft
power = None
penstock_length = 50 # ft
hp_type = 'Diversion'
headloss_method= "Hazen-Williams"
penstock_material = 'Concrete'

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length,
                           headloss_method= headloss_method, penstock_material= penstock_material)

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nFlow range evaluated (cfs):', np.round(hp.flow, 1))
print('Turbine Efficiency for the given flow range:', np.round(hp.effi_cal,3))
print('Power (kW) for the given flow range:', np.round(hp.power,1))

Design flow (cfs): 8000.0
Head_loss at design flow (ft): 2.0
Turbine type: Kaplan
Rated Power (Kw): 10885.57
Net head (ft): 18.0
Generator Efficiency: 0.98
Head Loss method: Hazen-Williams
Penstock length (ft): 50.0
Penstock diameter (ft): 12.76
Runner diameter (ft): 5.33

Flow range evaluated (cfs): [4800.  5082.4 5364.7 5647.1 5929.4 6211.8 6494.1 6776.5 7058.8 7341.2
 7623.5 7905.9 8188.2 8470.6 8752.9 9035.3 9317.6 9600. ]
Turbine Efficiency for the given flow range: [0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.91  0.908
 0.904 0.895 0.881 0.858 0.82  0.762]
Power (kW) for the given flow range: [ 6973.7  7351.9  7723.8  8089.8  8449.8  8803.6  9151.1  9491.7  9824.7
 10148.  10457.4 10744.6 10995.9 11188.7 11289.1 11248.  10996.6 10441.8]


In [9]:
# 2.4) Exploring additional options:
#  Using Hazen-Williams for head loss calculation and
# using a diffrent C value
# Note: editing hydraulic_processing.py allows for entering materials that can be called by name.

flow = 8000 # cfs
head = 20 # ft
power = None
penstock_length = 50 # ft
hp_type = 'Diversion'
headloss_method= "Hazen-Williams"
C = 100 # Hazen_williamns C

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length,
                           headloss_method= headloss_method, frictionfactor= C)

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nFlow range evaluated (cfs):', np.round(hp.flow, 1))
print('Turbine Efficiency for the given flow range:', np.round(hp.effi_cal,3))
print('Power (kW) for the given flow range:', np.round(hp.power,1))

Design flow (cfs): 8000.0
Head_loss at design flow (ft): 2.0
Turbine type: Kaplan
Rated Power (Kw): 10885.57
Net head (ft): 18.0
Generator Efficiency: 0.98
Head Loss method: Hazen-Williams
Penstock length (ft): 50.0
Penstock diameter (ft): 14.1
Runner diameter (ft): 5.33

Flow range evaluated (cfs): [4800.  5082.4 5364.7 5647.1 5929.4 6211.8 6494.1 6776.5 7058.8 7341.2
 7623.5 7905.9 8188.2 8470.6 8752.9 9035.3 9317.6 9600. ]
Turbine Efficiency for the given flow range: [0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.91  0.908
 0.904 0.895 0.881 0.858 0.82  0.762]
Power (kW) for the given flow range: [ 6973.7  7351.9  7723.8  8089.8  8449.8  8803.6  9151.1  9491.7  9824.7
 10148.  10457.4 10744.6 10995.9 11188.7 11289.1 11248.  10996.6 10441.8]


In [16]:
# 2.5) Head, power, and lenght of penstock are known. Multiple values of flow are available, the design flow is not known.
# In this scenario HydroGenerate will select a turbine, compute effieincy for the given flow values,
# design flow based on a percent of exceedance, penstock diameter (assuming steel if no material is given),
# head loss for all flows, rater power,
# power a given range of flow, 

flow = pd.read_csv('data_test.csv')['discharge_cfs'].to_numpy() # cfs
head = 20 # ft
power = None
penstock_length = 50 # ft
hp_type = 'Diversion'
pctime_runfull= 20 # percent of time the turbine is running full - default is 80%

# Note: decreasing the percent of time the turbine will run full will result in a 
# larger system (rated power and cost)

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length,
                           pctime_runfull= pctime_runfull)

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nFlow range evaluated (cfs):', np.round(hp.flow, 1))
print('Turbine Efficiency for the given flow range:', np.round(hp.effi_cal,3))
print('Power (kW) for the given flow range:', np.round(hp.power,1))

Design flow (cfs): 10800.0
Head_loss at design flow (ft): 1.88
Turbine type: Kaplan
Rated Power (Kw): 14828.83
Net head (ft): 18.12
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Penstock length (ft): 50.0
Penstock diameter (ft): 18.73
Runner diameter (ft): 6.14

Flow range evaluated (cfs): [3260. 3270. 3250. ... 3170. 3100. 3150.]
Turbine Efficiency for the given flow range: [0.768 0.77  0.766 ... 0.751 0.736 0.747]
Power (kW) for the given flow range: [4118.3 4140.3 4096.2 ... 3917.7 3759.3 3872.7]


In [19]:
# 2.6) Using flow as a pandas dataframe and adding annual energy calculation

# Note: When using a pandas dataframe as flow data, please set the datetime index before 
# using HydroGenerate.  (https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html)

flow = pd.read_csv('data_test.csv') # pandas data frame
flow['dateTime'] = pd.to_datetime(flow['dateTime']) # preprocessing convert to datetime
flow = flow.set_index('dateTime') # set datetime index # flolw is in cfs

head = 20 # ft
power = None
penstock_length = 50 # ft
hp_type = 'Diversion' 

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length,
                           flow_column= 'discharge_cfs', annual_caclulation= True)

pd.set_option('display.max_columns', 10) # 
pd.set_option('display.width', 1000)

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nPandas dataframe output: \n', hp.dataframe_output)
print('Annual output: \n', hp.annual_dataframe_output)

Design flow (cfs): 4120.0
Head_loss at design flow (ft): 1.82
Turbine type: Kaplan
Rated Power (Kw): 5629.86
Net head (ft): 18.18
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Penstock length (ft): 50.0
Penstock diameter (ft): 12.83
Runner diameter (ft): 3.89

Pandas dataframe output: 
                            discharge_cfs   site_id     power_kW  efficiency   energy_kWh
dateTime                                                                                
2010-01-01 08:00:00+00:00         3260.0  11370500  4621.331950    0.905788          NaN
2010-01-01 08:15:00+00:00         3270.0  11370500  4633.787900    0.905788  1158.446975
2010-01-01 08:30:00+00:00         3250.0  11370500  4608.860199    0.905788  1152.215050
2010-01-01 08:45:00+00:00         3270.0  11370500  4633.787900    0.905788  1158.446975
2010-01-01 09:00:00+00:00         3270.0  11370500  4633.787900    0.905788  1158.446975
...                                  ...       ...          ...         ...

In [42]:
# 2.7) Use streamflow data from a USGS stream gage. 
# HydroGenerate has functions to 
query = {}
query['id'] = '05487520'
query['start_date'] = '2010-01-01'
query['end_date'] = '2020-12-30'

# Call the get_data method. Note that the id_type has been set to 'sites'
flow_info = get_data(query, id_type='sites', path='../data_test_1.csv')

# Clean the data and make the date time column the index 
flow = clean_data(flow_info[0], column_change={'Streamflow, ft&#179;/s':'discharge_cfs'}, keep_nan=True, timestamp_col='dateTime', no_data=-999999)
flow.head()

https://waterservices.usgs.gov/nwis/iv/?format=json&sites=05487520&startDT=2010-01-01&endDT=2020-12-30&parameterCd=00060&siteStatus=all
Success with data retrieval from API


,discharge_cfs,qualifiers,site_name,site_id,lat,long
dateTime,,,,,,
2010-03-21 05:00:00+00:00,48400.0,[A],"Des Moines River near Swan, IA",05487520,41.489389,-93.278222
2010-03-23 05:00:00+00:00,44900.0,[A],"Des Moines River near Swan, IA",05487520,41.489389,-93.278222
2010-03-23 05:30:00+00:00,44900.0,[A],"Des Moines River near Swan, IA",05487520,41.489389,-93.278222
2010-03-23 06:00:00+00:00,44800.0,[A],"Des Moines River near Swan, IA",05487520,41.489389,-93.278222
2010-03-23 06:30:00+00:00,44800.0,[A],"Des Moines River near Swan, IA",05487520,41.489389,-93.278222


In [45]:
# 2.6) Using USGS streamflow data.

# The flow data for this example was created in the previous step. # cfs

head = 104 # ft
power = None
penstock_length = 20 # ft
hp_type = 'Diversion' 

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= hp_type, penstock_length= penstock_length,
                           flow_column= 'discharge_cfs', annual_caclulation= True)

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nPandas dataframe output: \n', hp.dataframe_output)
print('Annual output: \n', hp.annual_dataframe_output)

Design flow (cfs): 2780.0
Head_loss at design flow (ft): 9.23
Turbine type: Kaplan
Rated Power (Kw): 20369.2
Net head (ft): 94.77
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Penstock length (ft): 20.0
Penstock diameter (ft): 6.59
Runner diameter (ft): 3.23

Pandas dataframe output: 
                            discharge_cfs qualifiers                       site_name   site_id        lat       long     power_kW  efficiency  energy_kWh
dateTime                                                                                                                                                
2010-03-21 05:00:00+00:00        48400.0        [A]  Des Moines River near Swan, IA  05487520  41.489389 -93.278222    -0.000000    0.000000         NaN
2010-03-23 05:00:00+00:00        44900.0        [A]  Des Moines River near Swan, IA  05487520  41.489389 -93.278222    -0.000000    0.000000   -0.000000
2010-03-23 05:30:00+00:00        44900.0        [A]  Des Moines River near Swan, IA  05

In [94]:
# Full list of parameters included in HydroGenerate calculation
# dir(hp)

# Economic calculation

In [46]:
# 3.1) Cost and Annual O&M as part of a project. Resource_category =  'Non-PoweredDam'
flow = 8000 # cfs
head = 100 # ft
power = None

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= 'Diversion', penstock_length = 50,
                           resource_category= 'NewStream-reach')

# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Head_loss at design flow (ft):', round(hp.design_headloss, 2))
print('Turbine type:', hp.turbine_type)
print('Rated Power (Kw):', round(hp.rated_power, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Runner diameter (ft):', round(hp.runner_diameter,2))

print('\nResource Category:', hp.resource_category)
print('Initial Capital Cost (M$):', np.round(hp.icc,1))
print('Annual Operation and Maintennance (M$):', np.round(hp.annual_om,1))

Design flow (cfs): 8000.0
Head_loss at design flow (ft): 8.92
Turbine type: Kaplan
Rated Power (Kw): 56691.5
Net head (ft): 91.08
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Penstock length (ft): 50.0
Penstock diameter (ft): 12.17
Runner diameter (ft): 5.33

Resource Category: NewStream-reach
Initial Capital Cost (M$): 200.2
Annual Operation and Maintennance (M$): 2.0


In [47]:
# 3.2) resource_category: 'Non-PoweredDam'
head = 104 # ft
power = 20500 # kW

hp = calculate_hp_potential(rated_power= power, head= head,
                           hydropower_type= None, resource_category= 'Non-PoweredDam')

print('\nResource Category:', hp.resource_category)
print('Initial Capital Cost (M$):', np.round(hp.icc,1))
print('Annual Operation and Maintennance (M$):', np.round(hp.annual_om,1))


Resource Category: Non-PoweredDam
Initial Capital Cost (M$): 102.3
Annual Operation and Maintennance (M$): 1.2


In [49]:
# 3.3 Rewinding a generator. resource_caterogy= 'GeneratorRewind')
head = 104 # ft
power = 20500 # kW

hp = calculate_hp_potential(rated_power= power, head= head,
                           hydropower_type= None, resource_category= 'GeneratorRewind')

print('\nResource Category:', hp.resource_category)
print('Initial Capital Cost (M$):', np.round(hp.icc,1))
print('Annual Operation and Maintennance (M$):', np.round(hp.annual_om,1))


Resource Category: GeneratorRewind
Initial Capital Cost (M$): 3.0
Annual Operation and Maintennance (M$): 1.2


In [50]:
# Adding a new unit. resource_caterogy= 'UnitAddition')
head = 104 # ft
power = 20500 # kW

hp = calculate_hp_potential(rated_power= power, head= head,
                           hydropower_type= None, resource_category= 'UnitAddition')

print('\nResource Category:', hp.resource_category)
print('Initial Capital Cost (M$):', np.round(hp.icc,1))
print('Annual Operation and Maintennance (M$):', np.round(hp.annual_om,1))


Resource Category: UnitAddition
Initial Capital Cost (M$): 39.0
Annual Operation and Maintennance (M$): 1.0


In [55]:
# Annual energy generated and revenue
flow = 8000 # cfs
head = 20 # ft

hp = calculate_hp_potential(flow= flow, head= head, penstock_length= 50, 
                           hydropower_type= 'Diversion', # head_loss_calculation= False,
                           annual_caclulation= True, capacity_factor= 0.5)


# Explore output
print('Design flow (cfs):', hp.design_flow)
print('Turbine type:', hp.turbine_type)
print('Rated Power (MW):', round(hp.rated_power / 1000, 2))
print('Net head (ft):', round(hp.net_head, 2))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Runner diameter (ft):', round(hp.runner_diameter,2))


print('\nAnnual energy generated (MwH):', round(hp.annual_energy_generated / 1000, 0))
print('Number of days a year the plant operates:', hp.n_operation_days)
print('Annual revenue (M$):', round(hp.annual_revenue, 1))


Design flow (cfs): 8000.0
Turbine type: Kaplan
Rated Power (MW): 10.89
Net head (ft): 18.01
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Runner diameter (ft): 5.33

Annual energy generated (MwH): 42230.0
Number of days a year the plant operates: 182.5
Annual revenue (M$): 0.5


# Hydrokinetics Calculations

In [56]:
V = 2  # average cross-section velovicty m/s

hp = calculate_hp_potential(average_velocity = V,
                           hydropower_type = 'Hydrokinetic', units = 'SI',
                           hk_blade_type= 'ConventionalRotor')

print('Rated Power (kW):', round(hp.rated_power, 2))

Rated Power (kW): 0.46


In [57]:
V = 2 # average cross-section velovicty m/s

hp = calculate_hp_potential(average_velocity = V,
                           hydropower_type = 'Hydrokinetic', units = 'SI',
                           hk_blade_type= 'H-DarrieusRotor')

print('Rated Power (kW):', round(hp.rated_power, 2))

Rated Power (kW): 0.59


In [58]:
V = 2 # average cross-section velovicty m/s

hp = calculate_hp_potential(average_velocity = V,
                           hydropower_type = 'Hydrokinetic', units = 'SI',
                           hk_blade_type= 'DarrieusRotor')

print('Rated Power (kW):', round(hp.rated_power, 2))

Rated Power (kW): 0.38


In [59]:
V = 2 # average cross-section velovicty m/s

hp = calculate_hp_potential(average_velocity = V,
                           hydropower_type = 'Hydrokinetic', units = 'SI',
                           hk_blade_type= 'H-DarrieusRotor',
                           hk_blade_diameter= 1.36,
                           hk_blade_heigth = 1.8)

# hk_blade_diameter= None, 
#                            hk_blade_heigth= None, 
#                            hk_blade_type= None, 
#                            hk_swept_area = None):

print('Rated Power (kW):', round(hp.rated_power, 2))

Rated Power (kW): 1.44
